# Analyze metadata

The goal of this notebook is to analyze the available metadata:
- how many words and articles are available by search terms?
- how many once the blacklist has been applied
- how many after deduplication between search terms

In [1]:
import os
from pathlib import Path
from importlib import reload
from wikicollect.src import utils as ut
reload(ut)

<module 'wikicollect.src.utils' from '/Users/pierre/Documents/Workspace.nosync/general_projects/wikicollect/wikicollect/src/utils.py'>

#### Parameters

In [4]:
os.chdir(ut.find_repo_root())
metadata_folder = Path("wikicollect/data/metadata")
searches_folder = Path(metadata_folder, "searches")

#### Metadata loading

In [28]:
# get all metadata files available in metadata folder
searches_paths = [file for file in searches_folder.glob("*.yaml")] 
# load each one of them 
searches_results = [{search_result.parts[-1].replace(".yaml", ""): ut.load_yaml(search_result)} for search_result in searches_paths]
# get blacklist
blacklist = ut.load_yaml(Path(metadata_folder, "page_blacklist.yaml"))

#### Simple stats

In [57]:
print(f"We currently have {len(searches_results)} search term(s).\n")
for search_results in searches_results:
    search_term = list(searches_results[0].keys())[0]
    print(f"{search_term:=^60}")
    print(f"    > Number of search results: {len(search_results[search_term])}")
    total_words = sum([result["word_count"] for result in search_results[search_term]])
    print(f"    > Number of total words: {total_words: ,}")
    total_words_not_blacklisted = sum([result["word_count"] for result in search_results[search_term] if result["page_name"] not in blacklist[search_term]])
    print(f"    > Number of blacklisted pages: {len(blacklist[search_term])}")
    print(f"    > Number of total of non blacklisted words: {total_words_not_blacklisted:,}")
    print(f"    > Ratio of non-black listed words to total: {total_words_not_blacklisted / total_words: .2%}")

We currently have 1 search term(s).

=======================climate_change=======================
    > Number of search results: 500
    > Number of total words:  2,982,003
    > Number of blacklisted pages: 121
    > Number of total of non blacklisted words: 2,163,089
    > Ratio of non-black listed words to total:  72.54%


### Deduplication

In [43]:
#TODO: when we will have several search terms, add a deduplication step between page_ids.